In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install gym==0.12.1
!pip install -e git+https://dmsoroki@bitbucket.org/dmsoroki/gym_interf.git@master#egg=gym_interf
!pip install opencv-python

Obtaining gym_interf from git+https://dmsoroki@bitbucket.org/dmsoroki/gym_interf.git@master#egg=gym_interf
  Updating ./src/gym-interf clone (to revision master)
  Found existing installation: gym-interf 0.1
    Can't uninstall 'gym-interf'. No files were found to uninstall.
  Running setup.py develop for gym-interf
/bin/bash: pin: command not found


In [0]:
import gym
import gym_interf
import cv2

env = gym.make('interf-v1')

class Wrapper(gym.ObservationWrapper):
    def __init__(self, env):
        type(env).n_points = 256
        type(env).x_min = -4
        type(env).x_max = 4
        super().__init__(env)
    
    
    def observation(self, state):
        result = np.ndarray(shape=(16,64,64), dtype=np.uint8)
        for i, image in enumerate(state):
            resized = cv2.resize(image, (64, 64))
            result[i] = resized
        return result
    
env = Wrapper(env)


In [16]:
%matplotlib inline
import math
import random
import tqdm
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from collections import deque
from itertools import count
from copy import deepcopy
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T
from torch.distributions import Categorical

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display    
print("Is python : {}".format(is_ipython))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device : {}".format(device))


ACTIONS_NUM = 8
print("Number of actions : {}".format(ACTIONS_NUM))

Is python : True
Device : cuda
Number of actions : 8


In [0]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory:

    def __init__(self, capacity = 40000):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [0]:
class DQN(nn.Module):
    def __init__(self, in_channels=16, num_actions=ACTIONS_NUM):
        
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.conv4 = nn.Conv2d(64,1024,kernel_size=4,stride=1)
        self.advantage = nn.Linear(512, num_actions)
        self.value = nn.Linear(512, 1)

    def forward(self, x):
        x = x.float() / 255
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        advantage,value = torch.split(x,512,dim=1)
        
        advantage = advantage.view(advantage.shape[0],-1)
        value = value.view(value.shape[0],-1)
        
        advantage = self.advantage(advantage)
        value = self.value(value)
        q_value = value.expand(value.shape[0],ACTIONS_NUM) +\
        advantage-torch.mean(advantage,dim=1).unsqueeze(1).expand(advantage.shape[0],ACTIONS_NUM)
        return q_value

In [0]:
policy_net = DQN().to(device)

state = torch.load('drive/My Drive/colab/policy_net2')
policy_net.load_state_dict(state)
policy_net.eval()

target_net = DQN().to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer =optim.Adam(policy_net.parameters(),lr=1e-5)

memory = ReplayMemory()

def select_action(state, eps_threshold):
    global steps_done
    sample = random.random()
    if sample > eps_threshold:
        with torch.no_grad():
            state=state.float()
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(ACTIONS_NUM)]], device=device, dtype=torch.long)

train_rewards = []

mean_size = 100
mean_step = 1

def plot_rewards(rewards = train_rewards, name = "Train"):
    plt.figure(2)
    plt.clf()
    plt.title(name)
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(rewards)
    if len(rewards) > mean_size:
        means = np.array([rewards[i:i+mean_size:] for i in range(0, len(rewards) - mean_size, mean_step)]).mean(1)
        means = np.concatenate((np.zeros(mean_size - 1), means))
        plt.plot(means)

In [0]:
BATCH_SIZE = 32
GAMMA = 0.99

def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    
    
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))
    
    
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.uint8)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    
    state_batch=state_batch.float()
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    
    non_final_next_states=non_final_next_states.float()
    next_state_values = torch.zeros((BATCH_SIZE,1), device=device)
    next_state_actions = torch.zeros(BATCH_SIZE,dtype=torch.long, device=device)
    
    next_state_actions[non_final_mask] = policy_net(non_final_next_states).max(1)[1]
    next_state_values[non_final_mask] = target_net(non_final_next_states).gather(1, next_state_actions[non_final_mask].unsqueeze(1))
    next_state_values=next_state_values.squeeze(1)
    
    
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    
    
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1).detach())
    
    
    
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    
    del non_final_mask
    del non_final_next_states
    del state_batch
    del action_batch
    del reward_batch
    del state_action_values
    del next_state_values
    del expected_state_action_values
    del loss

In [0]:
def test():
  TEST_EPS = 0.005
  state = env.reset() 
  total_reward = 0
  for i in count():
    state = np.array(state,dtype=np.float32)
    state = torch.tensor(state, dtype=torch.float32, device=device)
    state = state.unsqueeze(0)
    action = select_action(state, TEST_EPS)
    state, _, done, info = env.step(action)
    reward = -1.0+info['visib']
    total_reward+=reward
    if done:
       break
  return i+1,total_reward,info['visib']
  

In [0]:
NUM_EPISODES = 200000


OPTIMIZE_MODEL_STEP = 4


TARGET_UPDATE=10000



STEPS_BEFORE_TRAIN = 30000



EPS_START = 1
EPS_END = 0.1
EPS_DECAY = 1000000

EPS_START_v2 = 0.1
EPS_END_v2 = 0.01

policy_net.train()
target_net.eval()
test_rewards = []


steps_done = 0

for e in range(NUM_EPISODES):

    state = env.reset() 
    state = np.array(state,dtype=np.float32)
    state = torch.tensor(state, dtype=torch.float32, device=device)
    state = state.unsqueeze(0)
    ep_rewards=0
    
    for t in range(180000):

        
        if steps_done<EPS_DECAY:
            if steps_done>STEPS_BEFORE_TRAIN:
                fraction=min(float(steps_done)/EPS_DECAY,1)
                eps_threshold= EPS_START + (EPS_END - EPS_START) * fraction
                action = select_action(state,eps_threshold)
            else:
                action=torch.tensor([[random.randrange(ACTIONS_NUM)]], device=device, dtype=torch.long)
        
        else:
            fraction=min(float(steps_done)/2*EPS_DECAY,1)
            eps_threshold= EPS_START_v2 + (EPS_END_v2 - EPS_START_v2) * fraction
            action = select_action(state,eps_threshold)
            
            
        
        next_state, _, done,info = env.step(action.item())
        reward = -1.0+info['visib']
        ep_rewards += reward
        
        next_state = np.array(next_state,dtype=np.float32)
        next_state = torch.tensor(next_state,dtype=torch.float32,device=device)
        next_state = next_state.unsqueeze(0)
        
        
        reward = torch.tensor([reward], device=device)
        if not done:
            memory.push(state, action,next_state, reward)
        else:
            next_state=None
            memory.push(state, action,next_state, reward)  
              
        steps_done+=1
        state=next_state
       
        
    
    
    
        if (steps_done > STEPS_BEFORE_TRAIN) and steps_done % OPTIMIZE_MODEL_STEP == 0:
            optimize_model()
        

        if steps_done % TARGET_UPDATE == 0:
            print("Target net updated!")
            target_net.load_state_dict(policy_net.state_dict())
            torch.save(policy_net.state_dict(),'curr_policy_net')
        

        if done:
            train_rewards.append(np.sum(ep_rewards))         
            plot_rewards()
            break 
    if e%100==0:
      policy_net.eval()
      total = 0
      val = 0 
      visib_end = 0.
      for _ in range(10):
        res0,res1,final_visib = test()
        val+= res0
        total += res1
        visib_end+=final_visib
      policy_net.train()
      print('---- steps_done {}  ---- Test_score {} ----- Number of steps needed {} --- final_visib = {}'.format(steps_done,total/10.,val/10.,visib_end/10.))

---- steps_done 200  ---- Test_score -2.102157699473312 ----- Number of steps needed 40.1 --- final_visib = 0.9963893907783223


In [0]:
torch.save(policy_net.state_dict(),'policy_net')

In [0]:
!kill -9 -1

In [0]:
# TEST_EPS = 0.005
 
dist_all = []
action_all = []
visib_all = []
steps = []
TEST_EPS = 0.0
env.reset_actions = 1000
env.max_steps = 200
for _ in range(100):
  state = env.reset()
  total_reward = 0
  for i in count():
    state = np.array(state,dtype=np.float32)
    state = torch.tensor(state, dtype=torch.float32, device=device)
    state = state.unsqueeze(0)
    action = select_action(state, TEST_EPS)
    state, _, done, info = env.step(action)
    reward = -1.0+info['visib']
    total_reward+=reward
    # dist_all.append(info['dist'])
    # visib_all.append(info['visib'])
    # action_all.append(action)
    if done:
        steps.append(i)
        break
    

In [0]:
steps = np.array(steps)
steps[steps<100].shape

In [0]:
np.sort(steps)

In [0]:
steps = []
dist_all = []


In [0]:
# TEST_EPS = 0.005
 
action_all = []
visib_all = []
TEST_EPS = 0.0
env.reset_actions = 5000
env.max_steps = 200
state = env.reset()
total_reward = 0
for i in count():
  state = np.array(state,dtype=np.float32)
  state = torch.tensor(state, dtype=torch.float32, device=device)
  state = state.unsqueeze(0)
  action = select_action(state, TEST_EPS)
  state, _, done, info = env.step(action)
  reward = -1.0+info['visib']
  total_reward+=reward
  if i == 0:
    dist_all.append(info['dist'])
  visib_all.append(info['visib'])
  action_all.append(action)
  if done:
    
    steps.append(i)
    break

In [0]:
plt.scatter(steps,dist_all)

In [0]:
plt.plot(dist_all)

In [0]:
plt.plot(visib_all)

In [0]:
visib_all[-1]